In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

import gc

# 1️⃣data load


- train

In [2]:
# parquet 파일 데이터를 읽어온다.
path = 'open/train/5.잔액정보/'

# 201807부터 201812까지의 데이터 파일을 하나의 데이터프레임에 저장
data_dict_train = {}
for i in range(7, 13):
    key = f'data{i:02d}'
    file_path = f'{path}2018{i:02d}_train_잔액정보.parquet'
    data_dict_train[key] = pd.read_parquet(file_path)
# 데이터프레임을 하나로 합친다.
df_train = pd.concat(data_dict_train.values(), ignore_index=True)
# 메모리 해제
gc.collect()
# 데이터프레임 확인
print(df_train.shape)

(2400000, 82)


- test

In [3]:
# parquet 파일 데이터를 읽어온다.
path = 'open/test/5.잔액정보/'

# 201807부터 201812까지의 데이터 파일을 하나의 데이터프레임에 저장
data_dict_test = {}
for i in range(7, 13):
    key = f'data{i:02d}'
    file_path = f'{path}2018{i:02d}_test_잔액정보.parquet'
    data_dict_test[key] = pd.read_parquet(file_path)
# 데이터프레임을 하나로 합친다.
df_test = pd.concat(data_dict_test.values(), ignore_index=True)
# 메모리 해제
gc.collect()
# 데이터프레임 확인
print(df_test.shape)

(600000, 82)


In [4]:
df_train.head(5)

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_6M,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,15988,2440,0,0,0,572,0,17008,0,0
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,7045,2677,0,2830,0,2736,0,0,0,0
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,66549,9118,0,8870,0,4429,0,43351,0,0
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,30139,884,0,0,0,5097,0,30697,0,0
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,28,21,0,0,0,0,0,0,0,0


# 2️⃣data preprocessing
### 💌notion
1. 카드론잔액_최종경과월(12) 제거 : 전부 0인 값임
2. 연체일자_B0M(13) 제거 : 거의 다 결측치임
3. 잔액_카드론은 최장기, 최단기만 남기고 제거(23~25)
4. RV_평균, 최대잔액은 최장기, 최단기만 남기고 제거(43,44)
5. 평잔은 최장기, 최단기만 남기고 제거(63~72)

- 결측치 확인

In [ ]:
# 데이터프레임 모든 컬럼에서 결측치 확인
# test에는 결측치 없음
missing_values = df_train.isnull().sum()
if missing_values.any():
    print("결측치가 있는 컬럼:")
    print(missing_values[missing_values > 0])

결측치가 있는 컬럼:
연체일자_B0M    2394336
dtype: int64


In [6]:
# 연체일자_B0M 컬럼 제거; 거의 다 결측치
df_train.drop(columns=['연체일자_B0M'], inplace=True)
df_test.drop(columns=['연체일자_B0M'], inplace=True)

- 값이 하나인 경우

In [7]:
# 데이터프레임 df에서 값이 하나(즉, 모든 값이 동일)인 컬럼 찾기
unique_one_cols = [col for col in df_train.columns if df_train[col].nunique() == 1]
print("값이 하나인 컬럼:", unique_one_cols)

값이 하나인 컬럼: ['카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_해외_B0M']


In [8]:
# 데이터 값이 하나인 컬럼 제거
df_train.drop(columns=unique_one_cols, inplace=True)
df_test.drop(columns=unique_one_cols, inplace=True)

In [9]:
# 잔액카드론 2,3,4개월 제거
df_train.drop(columns=['잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M'], inplace=True)
df_test.drop(columns=['잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M'], inplace=True)

In [10]:
# RV_평균잔액_R6M, RV_최대잔액_R6M
df_train.drop(columns=['RV_평균잔액_R6M', 'RV_최대잔액_R6M'], inplace=True)
df_test.drop(columns=['RV_평균잔액_R6M', 'RV_최대잔액_R6M'], inplace=True)

In [11]:
# 평잔 3개월 모두 제거
df_train.drop(columns=['평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M',
                       '평잔_RV일시불_3M', '평잔_RV일시불_해외_3M',
                       '평잔_할부_3M', '평잔_할부_해외_3M',
                       '평잔_CA_3M', '평잔_CA_해외_3M',
                       '평잔_카드론_3M'], inplace=True)
df_test.drop(columns=['평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M',
                     '평잔_RV일시불_3M', '평잔_RV일시불_해외_3M',
                     '평잔_할부_3M', '평잔_할부_해외_3M',
                     '평잔_CA_3M', '평잔_CA_해외_3M',
                     '평잔_카드론_3M'], inplace=True)

In [12]:
len(df_train.columns), len(df_test.columns)

(58, 58)

# 3️⃣data store

In [13]:
100/0

ZeroDivisionError: division by zero

In [14]:
# df_train을 parquet 파일로 저장
df_train.to_parquet('train_잔액정보.parquet', index=False)

In [15]:
df_test.to_parquet('test_잔액정보.parquet', index=False)